In [ ]:
from __future__ import division
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import copy
import random

N =  17                 # Number of nodes
rw = [500,700,1150]      # Array with Wealth values of rich
pw = [500,300,200]       # Array with Wealth values of poor
pr = [0.5,0.5,0.3]       # Array with Probabilities of being rich
b_arr = [0.1,0.1,0.1]    # Array storing b-values for 3 different initial GINI case
M = 20                  # Number of rounds
co = 50                 # Unitary Cost of co-operation
be = 100                # Unitary Benefit of co-operation
re = 0.3                # Rewiring probability
S = 1000                 # Number of iterations
pl = 0.3                # Initial probability of a link being formed between any two nodes

def inv_logit(y):
    inv_logit_out =(np.exp(y)/(1+np.exp(y)))
    return inv_logit_out

def gini_calc(x):       #IN: Wealth values across the population, OUT: Gini Coefficient
    sum2=0
    for i in range(len(x)):
        sum1=0
        for j in range(len(x)):
            sum1=sum1+np.abs(x[i]-x[j])
        sum2=sum2+sum1;
    
    gini=sum2/(2*len(x)**2*np.mean(x));
    return gini

node_arr = np.arange(N)
result_array = np.zeros([3,S,M+1])        # Storing GINI coefficients in various rounds over different iterations
deg_array = np.zeros([3,S,M+1])            # Storing mean degrees in various rounds over different iterations
avg_wealth_array = np.zeros([3,S,M+1])    # Storing average wealth in various rounds over different iterations
co_op_array =  np.zeros([3,S,M])        # Storing cooperator fraction in various rounds over different iterations
flag = 0

for h1 in range(3):   
    for h in range(S):       
        G = nx.erdos_renyi_graph(N, pl)
        A = nx.adjacency_matrix(G)
        E = G.number_of_edges()
        No_of_pairs = (N*(N-1))/2
        No_of_rewiring = round(No_of_pairs*re)
        Nre = int(No_of_rewiring)        
        init_wealth_array = np.random.choice([rw[h1], pw[h1]],N,p=[pr[h1],1-pr[h1]])      
        GINI = gini_calc(init_wealth_array)   
        
        result_array[h1, h, 0] = GINI          
        deg_array[h1, h, 0] = float((2*E)/N)
        avg_wealth_array[h1, h, 0] = np.average(init_wealth_array)        
    
        for k in range(M):                 # Continuing the game for M rounds            
            if (k==0):                    # First round 
                p1 = inv_logit((-1.017021)*GINI + (0.8130213))  # functional form taken from Christakis paper for comparison               
                decision_array = np.random.choice([1,0],N,p=[p1,1-p1])   # Decision making in first round                 
                
                for i in range(N):         # For all nodes in the network
                    nn = G.degree(i)                                                
                    # Wealth update   
                    if (decision_array[i] == 1): 
                        init_wealth_array[i] = init_wealth_array[i] - co*(nn)   # Updatation of wealth of a cooperative node
                        for l in range(N):          	# Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue
                    else:
                        continue               
                
            else:                     # Decision making in subsequent rounds
                new_decision_array = np.zeros(N)                
                for i in range(N):         # For all nodes in the network
                    nn = G.degree(i)                                    
                    cop = 0                     # Initializing the no of cooperator neighbours
                    for j in range(N):
                        if (A[i,j] == 1):               # If a link exists between nodes i and j
                            if (decision_array[j] == 1):        # If link j cooperated in last round
                                cop += 1
                            elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                                continue
                        else:
                            continue
                
                    sum_neighbor_wealth = 0
                    for j in range(N):			# Total wealth of neighbors
                        if (A[i,j] == 1):
                            sum_neighbor_wealth += init_wealth_array[j]
                        else:
                            continue
                 
                    if (nn>0):
                        av_neighbor_wealth = (sum_neighbor_wealth)/nn
                        diff_wealth = av_neighbor_wealth - (init_wealth_array[i])
                        pc = p1 + (1-p1)*(np.tanh(0.001*diff_wealth)) - b_arr[h1]
                    else:
                        pc = p1                   
                
                
                    if (nn>0):              # For postive number of nearest neighbours
                        cop_rat = float(cop/nn)                    
                        if (cop_rat < 0.5):       # If cooperation parameter is lesser than 0.5 i.e. selfish environment                           
                            c = np.random.random()
                            if (c < pc):
                                new_decision_array[i] = 1       # Node-i decides to be cooperative                          
                            else:
                                new_decision_array[i] = 0

                        elif (cop_rat > 0.5):          # Cooperation parameter is greater than 0.5 i.e. cooperative environment                            
                            c = np.random.random()
                            if (c < pc):                        
                                new_decision_array[i] = 0
                            else:
                                new_decision_array[i] = 1
                            
                        elif (cop_rat == 0.5):          # neutral environment                   
                              c = np.random.random()
                              if (c>=pc):
                                  new_decision_array[i] = 1
                              else:
                                  new_decision_array[i] = 0
                                    
                    else:
                        new_decision_array[i] = 0                        
                for i in range(N):
                    if (new_decision_array[i] == 1): 
                        init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))   # Updatation of wealth of a cooperative node
                        for l in range(N):          	# Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue
                    else:
                        continue    
                               
                decision_array = copy.copy(new_decision_array)         
        
            co_op_array[h1,h,k] = np.average(decision_array)
            avg_wealth_array[h1,h,k+1] = np.average(init_wealth_array)
            result_array[h1,h,k+1] = gini_calc(init_wealth_array)
            
            # Rewiring step
                  
            temp_array = []                        
                                                                                     
            while (len(temp_array) < Nre) :            
                i = random.choice(node_arr)
                j = random.choice(node_arr)
                    
                if (i==j):
                    continue
                else:
                
                    if (i,j) in temp_array:
                        continue
                    elif (j,i) in temp_array:
                        continue
                    else:
                        temp_array.append((i,j))
                        temp = [i,j]
                                                             
                        if (A[i,j] == 1):                              # If a link exists between the selected pair
                            b = random.choice(temp)
                            
                            if (b == i):                               # If node-i is given the choice to break or not
                                if (decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 0.7):
                                    
                                        G.remove_edge(i,j)                 # Breakage of link for node-i disagreeing
                                  
                                    else:
                                        continue                        # Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 0.87):
                                    
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                            else:                                   # If node-j is given the choice to break or not	
                                if (decision_array[i] == 0):
                                    c = np.random.random()
                                    if (c < 0.7):
                                    
                                        G.remove_edge(i,j)         # Breakage of link for node-j disagreeing
                                    else:
                                        continue			# Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 0.87):
                                        
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                                                                
            
                        else:                 # If a link does not exist between the selected pair
                            if (decision_array[i] == 1 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 0.93):
                                
                                    G.add_edge(i,j)
                                else:
                                    continue
                            
                            if (decision_array[i] == 0 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 0.80):
                                
                                    continue
                                else:
                                    G.add_edge(i,j)
                        
                            if (decision_array[i] == 0 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 0.70):
                                
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
                            if (decision_array[i] == 1 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 0.70):
                                
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
            A = nx.adjacency_matrix(G)
            deg_array[h1,h,k+1] = float((2*G.number_of_edges())/N)                     
        
        flag += 1
        yo = (flag*100)/(3*S)
        print('Job completion percentage: %f' %yo)
        
np.savez('visible_invert_pc',result_array=result_array,co_op_array=co_op_array,avg_wealth_array=avg_wealth_array,deg_array=deg_array)